In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/MyDrive/dacon_anomaly')

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

In [ ]:
train_df = pd.read_csv('./data/train.csv') # Train
train_x = train_df.drop(columns=['ID'])

val_df = pd.read_csv('./data/val.csv') # Validation
val_x = val_df.drop(columns=['ID', 'Class']) # Input Data
val_y = val_df['Class'] # Label

val_normal, val_fraud = val_df['Class'].value_counts()
val_contamination = val_fraud / val_normal

In [ ]:
# Singular Value Decomposition

from sklearn.decomposition import TruncatedSVD

n_components = 20
algorithm = 'randomized'
random_state = 42

svd = TruncatedSVD(n_components=n_components, algorithm=algorithm, random_state=random_state)

X_train_svd = svd.fit_transform(train_x)
X_train_svd = pd.DataFrame(data=X_train_svd)

X_valid_svd = svd.transform(val_x)
X_valid_svd = pd.DataFrame(data=X_valid_svd)

- One-class support vector machine  
  : 주어진 데이터를 잘 설명할 수 있는 최적의 support vector를 구하고 이 영역 밖의 데이터들은 outlier로 간주하는 방식
  ![image.png](https://blog.kakaocdn.net/dn/Rw9bH/btqNiVj5r0j/hGDHojo9Nk7IakWnbjAIdK/img.png)

In [ ]:
# One-Class svm
from sklearn.svm import OneClassSVM

clf = OneClassSVM(nu = 0.001, kernel = "rbf", gamma = 0.0001)
clf.fit(X_train_svd)

## nu = 0.001, gamma = 0.0001, kernel = 'rbf'  => 0.7034

OneClassSVM(gamma=0.0001, nu=0.001)

In [ ]:
### evaluation : validation set

def get_pred_label(model_pred):
    model_pred = np.where(model_pred == 1, 0, model_pred)
    model_pred = np.where(model_pred == -1, 1, model_pred)
    return model_pred

In [ ]:
val_pred = clf.predict(X_valid_svd) # model prediction
val_pred = get_pred_label(val_pred)
val_score = f1_score(val_y, val_pred, average='macro')
print(f'Validation F1 Score : [{val_score}]')
print(classification_report(val_y, val_pred))

Validation F1 Score : [0.6997098595016618]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     28432
           1       0.44      0.37      0.40        30

    accuracy                           1.00     28462
   macro avg       0.72      0.68      0.70     28462
weighted avg       1.00      1.00      1.00     28462



In [ ]:
test_df = pd.read_csv('./data/test.csv') # Train
test_x = test_df.drop(columns=['ID'])

In [ ]:
X_test_svd = svd.transform(test_x)
X_test_svd = pd.DataFrame(data=X_test_svd)

In [ ]:
test_pred = clf.predict(X_test_svd) # model prediction
test_pred = get_pred_label(test_pred)

In [ ]:
submit = pd.read_csv('./data/sample_submission.csv')

In [ ]:
submit['Class'] = test_pred
submit.to_csv('submit_SVDOneClassSVM.csv', index=False)